In [1]:
import OpenDartReader
import pandas as pd

dart = OpenDartReader("4925a6e6e69d8f9138f4d9814f56f371b2b2079a")

In [2]:
ticker="000370"

In [3]:
sorted_df = dart.list(ticker, start='2023-01-01', kind='A').sort_values('rcept_dt', ascending=True)

In [4]:
from segment_sales_crawling.financial_report_analyzer_soomin import ReportFilter

filtered_df = ReportFilter.filter_annual_reports(sorted_df)

In [5]:
import re

oldest_report = filtered_df.iloc[0]['report_nm']
oldest_year = int(re.search(r'\((\d{4})', oldest_report).group(1))
oldest_month = int(re.search(r'\.(\d{2})', oldest_report).group(1))

In [6]:
latest_report = filtered_df.iloc[-1]['report_nm']
latest_year = int(re.search(r'\((\d{4})', latest_report).group(1))
latest_month = int(re.search(r'\.(\d{2})', latest_report).group(1))

In [7]:
# latest_month 에 따라 이전 년도 및 현재 년도의 추출할 보고서 결정
report_code_dict = {
    3: [11013,11012,11014,11011,11013], # 3월일 경우 : 1분기, 반기, 3분기, 사업, 1분기
    6: [11012,11014,11011,11013,11012], # 6월일 경우 : 반기, 3분기, 사업, 1분기, 반기
    9: [11014,11011,11013,11012,11014], # 9월일 경우 : 3분기, 사업, 1분기, 반기, 3분기
    12: [11011,11013,11012,11014,11011], # 12월일 경우 : 사업, 1분기, 반기, 3분기, 사업
    }

In [8]:
def str_to_int(value):
    if isinstance(value, str):
        return int(value.replace(',', ''))
    return value

In [10]:
n = 5-latest_month//3 
# 9월 = 3분기 -> 5-3 = 2 -> 3분기,4분기 = 이전 년도 2개 분기값 필요
# 6월 = 2분기 -> 5-2 = 3 -> 2분기,3분기,4분기 = 이전 년도 3개 분기값 필요
# 3월 = 1분기 -> 5-1 = 4 -> 1분기,2분기,3분기,4분기 = 이전 년도 4개 분기값 필요
# 12월 = 4분기 -> 5-4 = 1 -> 4분기 = 이전 년도 1개 분기값 필요

save_values_dict = {}

def get_financial_data(df:pd.DataFrame, report_type:str) -> list[int]:
    """재무제표에서 매출액, 영업이익, 당기순이익을 추출하는 함수"""
    amount_column = 'thstrm_amount' if report_type == '11011' else 'thstrm_add_amount' # 11011(사업보고서의 경우에만 누적 매출, 영업이익, 당기순이익 컬럼명이 다름)
    
    def get_value(account_name:str) -> int:
        return str_to_int(df.loc[(df['account_nm'] == account_name) & 
                                (df['fs_nm'] == '연결재무제표')][amount_column].iloc[0])
    
    return [
        get_value('매출액'),
        get_value('영업이익'),
        get_value('당기순이익')
    ]
# 이전 년도 분기의 매출, 영업이익, 당기순이익 추출
for i in report_code_dict[latest_month][:n]:
    df = dart.finstate(ticker, oldest_year, reprt_code=str(i))
    financial_data = get_financial_data(df, str(i))
    save_values_dict[f"{oldest_year}-{i}"] = financial_data

IndexError: single positional indexer is out-of-bounds

In [16]:
ticker="035720"
df = dart.finstate(ticker, 2023, reprt_code="11014")
df

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,ord,currency,thstrm_add_amount,frmtrm_add_amount
0,20231226000387,11014,2023,00258801,035720,CFS,연결재무제표,BS,재무상태표,유동자산,제 29 기3분기말,2023.09.30 현재,"8,695,272,771,351",제 28 기말,2022.12.31 현재,"7,755,732,886,687",1,KRW,NaN,NaN
1,20231226000387,11014,2023,00258801,035720,CFS,연결재무제표,BS,재무상태표,비유동자산,제 29 기3분기말,2023.09.30 현재,"16,936,067,243,005",제 28 기말,2022.12.31 현재,"14,450,930,891,040",3,KRW,NaN,NaN
2,20231226000387,11014,2023,00258801,035720,CFS,연결재무제표,BS,재무상태표,자산총계,제 29 기3분기말,2023.09.30 현재,"26,785,564,868,322",제 28 기말,2022.12.31 현재,"22,963,491,575,614",5,KRW,NaN,NaN
3,20231226000387,11014,2023,00258801,035720,CFS,연결재무제표,BS,재무상태표,유동부채,제 29 기3분기말,2023.09.30 현재,"6,696,232,633,124",제 28 기말,2022.12.31 현재,"5,148,326,794,720",7,KRW,NaN,NaN
4,20231226000387,11014,2023,00258801,035720,CFS,연결재무제표,BS,재무상태표,비유동부채,제 29 기3분기말,2023.09.30 현재,"3,629,598,427,216",제 28 기말,2022.12.31 현재,"3,843,929,114,057",9,KRW,NaN,NaN
5,20231226000387,11014,2023,00258801,035720,CFS,연결재무제표,BS,재무상태표,부채총계,제 29 기3분기말,2023.09.30 현재,"11,124,868,765,051",제 28 기말,2022.12.31 현재,"9,431,629,284,734",11,KRW,NaN,NaN
6,20231226000387,11014,2023,00258801,035720,CFS,연결재무제표,BS,재무상태표,자본금,제 29 기3분기말,2023.09.30 현재,"44,498,544,400",제 28 기말,2022.12.31 현재,"44,592,368,900",13,KRW,NaN,NaN
7,20231226000387,11014,2023,00258801,035720,CFS,연결재무제표,BS,재무상태표,이익잉여금,제 29 기3분기말,2023.09.30 현재,"3,166,163,167,804",제 28 기말,2022.12.31 현재,"3,046,458,208,012",17,KRW,NaN,NaN
8,20231226000387,11014,2023,00258801,035720,CFS,연결재무제표,BS,재무상태표,자본총계,제 29 기3분기말,2023.09.30 현재,"15,660,696,103,271",제 28 기말,2022.12.31 현재,"13,531,862,290,880",21,KRW,NaN,NaN
9,20231226000387,11014,2023,00258801,035720,CFS,연결재무제표,IS,손익계산서,영업이익,제 29 기3분기,2023.01.01 ~ 2023.09.30,"140,296,343,036",제 28 기3분기,2022.01.01 ~ 2022.09.30,"150,331,716,201",25,KRW,"324,894,046,533","480,009,098,456"


In [9]:
# 현재 년도 분기의 매출, 영업이익, 당기순이익 추출
for i in report_code_dict[latest_month][n:5]:
    df = dart.finstate(ticker, latest_year, reprt_code=str(i))
    financial_data = get_financial_data(df, str(i))
    save_values_dict[f"{latest_year}-{i}"] = financial_data

NameError: name 'n' is not defined

In [12]:
save_values_dict

{'2023-11014': [13081220000000, 1456149000000, 958495000000],
 '2023-11011': [17608511000000, 1753204000000, 1145937000000],
 '2024-11013': [4474611000000, 498489000000, 361940000000],
 '2024-11012': [8896970000000, 1035997000000, 712114000000],
 '2024-11014': [13429088000000, 1569260000000, 992290000000]}

In [13]:
df = pd.DataFrame(save_values_dict).T.transpose()
new_column_name = "계정"
df.insert(0, new_column_name, ['영업수익', '영업이익', '순이익'])

In [14]:
# 컬럼명 변경을 위한 매핑 딕셔너리 생성
quarter_mapping = {
    '11014': '3Q',
    '11011': '4Q',
    '11013': '1Q',
    '11012': '2Q'
}

# 컬럼명 변경
for col in df.columns[1:]:  # '계정' 컬럼을 제외한 나머지 컬럼들에 대해
    year = col.split('-')[0]
    quarter_code = col.split('-')[1]
    new_col = f"{year}-{quarter_mapping[quarter_code]}"
    df = df.rename(columns={col: new_col})

df

,계정,2023-3Q,2023-4Q,2024-1Q,2024-2Q,2024-3Q
0,영업수익,13081220000000,17608511000000,4474611000000,8896970000000,13429088000000
1,영업이익,1456149000000,1753204000000,498489000000,1035997000000,1569260000000
2,순이익,958495000000,1145937000000,361940000000,712114000000,992290000000


In [15]:
df.to_excel(f"/Users/jaahyun/valuation_agent/report_agent/data/quarterly_financial_data/{ticker}_quarterly_financial_data.xlsx", index=False)